<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>


# **Hyperparameter Optimization for Keras with Scikit-Learn**


Estimated time needed: **35** minutes


We already know how to use `RandomizedSearchCV` and `GridSearchCV` for hyperparameter tuning in machine learning models - linear regression,  decision trees, and so on. It turns out that we can utilize the same functionality easily for neural networks! Keras offers a scikit-learn wrapper that lets us perform randomized/grid search on its models using the same syntax (example `fit()`, `.best_score_`). In this lab, we will take a look at how to do so for a Sequential model.

As a reminder, both search types may take a long time to run for this lab.


## **Table of Contents**

<ol>
    <li><a href="https://#Objectives">Objectives</a></li>
    <li>
        <a href="https://#Setup">Setup</a>
        <ol>
            <li><a href="https://#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="https://#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="https://#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li>
        <a href="https://#Create-the-Model">Create the Model</a>
        <ol>
            <li><a href="https://#Load-the-Data">Load the Data</a></li>
            <li><a href="https://#Data-Wrangling">Data Wrangling</a></li>
            <li><a href="https://#Build-the-Base-Model">Build the Base Model</a></li>
        </ol>
    </li>  
    <li>
        <a href="https://#Randomized-Search">Randomized Search</a>
        <ol>
            <li><a href="https://#Parameters">Parameters</a></li>
            <li><a href="https://#Define-and-Fit-RandomizedSearchCV">Define and Fit RandomizedSearchCV</a></li>
            <li><a href="https://#Performance-Evaluation">Performance Evaluation</a></li>
        </ol>
    </li> 
    <li>
        <a href="https://#Exercised">Exercises</a>
        <ol>
            <li><a href="https://#Exercise-1:-Build-the-Base-Model">Exercise 1: Build the Base Model</a></li>
            <li><a href="https://#Exercise-2:-Define-Search-Parameters">Exercise 2: Define Search Parameters</a></li>
            <li><a href="https://#Exercise-3:-Fit-RandomizedSearchCV">Exercise 3: Fit RandomizedSearchCV</a></li>
        </ol>
    </li>    
</ol>


## Objectives

After completing this lab you will be able to:

*   Use Keras' scikit-learn wrapper to utilize sklearn functions on Keras models
*   Apply randomized search on Keras models to find the best hyperparameters


***


## Setup


For this lab, we will be using the following libraries:

*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and machine-learning-pipeline related functions.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.
*   [`tensorflow`](https://www.tensorflow.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and neural network related functions.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run this notebook command in a different Jupyter environment (like Watson Studio or Anaconda), you will need to install these libraries by removing the `#` sign before `!mamba` in the following code cell.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy numpy==1.21.4 matplotlib==3.5.0 scikit-learn==0.20.1
# Note: If your environment doesn't support "!mamba install", use "!pip install"

The following required libraries are **not** pre-installed in the Skills Network Labs environment. **You will need to run the following cell** to install them:


In [ ]:
#!mamba install -qy tqdm

In [ ]:
# Upgrade to newest version of skillsnetwork for more functionalities
# Restart kernel after doing so
!pip install --upgrade skillsnetwork

### Importing Required Libraries


In [1]:
%pip uninstall tensorflow
%pip install tensorflow

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # tensorflow INFO and WARNING messages are not printed
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from tqdm import tqdm
import numpy as np
%matplotlib inline

import tensorflow as tf
import keras
# import scikeras
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
# from scikeras.wrappers import KerasClassifier

import skillsnetwork

ModuleNotFoundError: No module named 'keras.saving.legacy'

### Defining Helper Functions


In [3]:
# Vectorize integer sequence
def vectorize_sequence(sequence, dimensions):
    results = np.zeros((len(sequence), dimensions))
    for index,value in enumerate(sequence):
        if max(value) < dimensions:
            results[index, value] = 1
    return results

# Convert label into one-hot format
def one_hot_label(labels, dimensions):
    results = np.zeros((len(labels), dimensions))
    for index,value in enumerate(labels):
        if value < dimensions:
            results[index, value] = 1
    return results

## Create the Model


### Load the Data


For this exercise, we will be using the [Reuters newswire classification dataset](https://keras.io/api/datasets/reuters/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01) from Keras. The training features for this dataset are lists of word indices (integers), corresponding to their frequency in the dataset. The response labels take on one of 46 classes, representing the newswire's topic.


In [5]:
await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module2/L1/reuters.npz", overwrite=True)


FileNotFoundError: [Errno 2] No such file or directory: '\\tmp\\skills-network-395185324659035057-reuters.npz'

In [6]:
X = np.load("x.npy", allow_pickle=True)
y = np.load ("y.npy", allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

To get the word for a specific index, we can also extract a dictionary of words to index using the following Keras function.


In [7]:
word_to_ind = tf.keras.datasets.reuters.get_word_index(path="reuters_word_index.json")

550378/550378 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Data Wrangling


Since each observation is a list of words that appear in the newswire, the length varies. Hence, we will vectorize the dataset using `vectorize_sequence()` to ensure that all inputs to our model have the same dimension. Labels are also one-hot encoded with `one_hot_label()` because classes (news topic) are not ordinal.


In [8]:
dim_x = max([max(sequence) for sequence in X_train])+1
dim_y = max(y_train)+1

X_train_vec = vectorize_sequence(X_train, dim_x)
X_test_vec = vectorize_sequence(X_test, dim_x)
y_train_hot = one_hot_label(y_train, dim_y)
y_test_hot = one_hot_label(y_test, dim_y)

### Build the Base Model


In order to apply `RandomizedSearchCV` on Keras models, we will be using `KerasClassifier` from `keras.wrappers.scikit_learn` library, which will let us apply scikit-learn functions on the model.


We define `create_model()` below to detail which layers we want to include in the model. Recall that the final Dense layer has 46 units to correspond to the number of classes. This also prompts us to use categorical cross entropy as a loss function. Here, `neuron` is included as a parameter with default value because we want to tune it later.


In [10]:
# Create Keras Sequential Model as base model
def create_model(neurons = 10):
    model = Sequential()
    model.add(Dense(neurons, activation='linear'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(46, activation='softmax'))
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

For the base model, we won't change any parameters so that we can compare them with results after hyperparameter tuning. We also specify some of the default values for hyperparameters that don't appear in `create_model()` (example batch_size, epochs) such that they are defined when applying randomized search.


In [11]:
np.random.seed(0)
base_model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=10, epochs=1)

Fitting the model on the train set, we obtain the test score for our base model.


In [15]:
# Get pre-tuned results
base_model.fit(X=X_train_vec, y=y_train_hot)
base_score = base_model.score(X_test_vec, y_test_hot)
print("The baseline accuracy is: %.3f" % base_score)


ValueError: Sequential model 'sequential_2' has no defined outputs yet.

## Randomized Search


### Parameters


As you might already know from performing randomized search on machine learning models, we have to create a dictionary for the hyperparameter values. Let's start by defining the values we want to experiment with! Note that if you would like to test other parameters, they must be defined in the base model as well.


In [ ]:
batch_size = [10, 20, 60, 80]
epochs = [1, 3, 5]
neurons = [1, 10, 20, 30]

params = dict(batch_size=batch_size, epochs=epochs, neurons=neurons)
params

### Define and Fit RandomizedSearchCV


In [ ]:
search = RandomizedSearchCV(estimator=base_model, param_distributions=params, cv=3)


Now, fit randomized search on `X_train_vec` and `y_train_hot` as you would for any other model. **Note that this may take a while to run (10+ minutes)**, especially if there are a lot of parameter combinations, or if the epoch size is big. If you have the resources, you could also switch out `RandomizedSearchCV` for `GridSearchCV` to search over every combination of hyperparameters (takes even more time to run).


In [ ]:
search_result = search.fit(X_train_vec, y_train_hot)


### Performance Evaluation


Let's take a look at the results from this search! In particular, we will examine the mean and standard deviation of the cross-validation score under different hyperparameter combinations.


In [ ]:
means = search_result.cv_results_['mean_test_score']
stds = search_result.cv_results_['std_test_score']
params = search_result.cv_results_['params']

`RandomizedSearchCV` also has attributes for us to access the best score and parameters directly.


In [ ]:
print("Best mean cross-validated score: {} using {}".format(round(search_result.best_score_,3), search_result.best_params_))


We can also print out all the other scores:


In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print("Mean cross-validated score: {} ({}) using: {}".format(round(mean,3), round(stdev,3), param))

From this, we can see how different the other models' scores are compared to the optimal model's performance. Some are pretty close to the best score, whereas there are combinations that yield much lower scores.Thank goodness we didn't pick those! With randomized search on neural networks, we are able to determine the best values in an automated way.


Using the best estimator, let's get the test score:


In [ ]:
print("Best test score: %.3f" % search_result.best_estimator_.score(X_test_vec, y_test_hot))

Our test score has increased compared to the base model!


## Exercises


Now, let's try Randomized search on other hyperparameters!


### Exercise 1: Build the Base Model


This time, we want to look at different optimizers, learning rates, and dropout rates. Since the learning rate is a value fed into `optimizer`, the parameter has to be named a certain way in `create_model()` and `params`: `optimizer__learning_rate`.


In [ ]:
# Create Keras Sequential Model as base model
def create_model(optimizer = 'RMSprop', optimizer__learning_rate = 0.1, dropout_rate = 0.2):
    model = Sequential()
    model.add(Dense(64, activation='linear'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(46, activation='softmax'))
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# TODO: Create `base_model` using `KerasClassifier`. Fit it to the training set and obtain the test score


<details>
    <summary>Click here for sample solution</summary>

```python

np.random.seed(0)
base_model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=100, epochs=1)
base_model.fit(X_train_vec, y_train_hot)
base_score = base_model.score(X_test_vec, y_test_hot)
print("The baseline accuracy is: {}".format(base_score))

```

</details>


### Exercise 2: Define Search Parameters


Now, we will specify which values we want to experiment with and put them into a dictionary.


In [ ]:
# TODO: specify 3-4 values for each of the following parameters

# TODO: convert it into a dictionary variable named `params`


<details>
    <summary>Click here for sample solution</summary>

```python

optimizer = ['SGD','RMSprop','Adam']
learning_rate = [0.01, 0.1, 1]
dropout_rate = [0.1, 0.3, 0.6, 0.9]
params = dict(optimizer=optimizer, optimizer__learning_rate=learning_rate, dropout_rate = dropout_rate)

```

</details>


### Exercise 3: Fit RandomizedSearchCV **(Note that this may take a while to run (5+ minutes))**


In [ ]:
# TODO: Create `RandomizedSearchCV` object `search` and fit it to the train set


<details>
    <summary>Click here for solution</summary>

```python

search = RandomizedSearchCV(estimator=base_model, param_distributions=params, cv=3)
search_result = search.fit(X_train_vec, y_train_hot)

```

</details>


In [ ]:
# TODO: Obtain the best cross-validation score and test scores


<details>
    <summary>Click here for solution</summary>

```python

print("Best mean cross-validated score: {} using {}".format(round(search_result.best_score_,3), search_result.best_params_))
print("Best test score: %.3f" % search_result.best_estimator_.score(X_test_vec, y_test_hot))

```

</details>


## Authors


[Cindy Huang](https://www.linkedin.com/in/cindy-shih-ting-huang/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01) is a data science associate of the Skills Network team. She has a passion for machine learning to improve user experience, especially in the area of computational linguistics.


### Other Contributors


[Contributor with Link](contributor_linl), Contributor No Link


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description |
| ----------------- | ------- | ---------- | ------------------ |
| 2022-07-20        | 1.0     | Cindy H.   | Create lab draft   |
| 2022-09-01        | 1.0     | Steve Hord | QA pass edits      |
| 2022-11-11        | 1.0     | Shengkai C.| Review and edit    |


Copyright © 2022 IBM Corporation. All rights reserved.
